<a href="https://colab.research.google.com/github/rajath02/classifying-social-meadia-comments/blob/main/csmm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/data_preprocessing/df_merged.csv')

dataset = dataset.fillna(0)
dataset

,Comment,good,discussion,motivational,demotivational,irrelavent or abuse
0,"""You fuck .""",0.0,0.0,0.0,0.0,1.0
1,"""i really don't understand your point.\xa0 It ...",0.0,1.0,0.0,0.0,0.0
2,"""A\\xc2\\xa0majority of Canadians can and has ...",0.0,0.0,0.0,0.0,1.0
3,"""listen if you dont wanna get married to a man...",0.0,0.0,0.0,0.0,1.0
4,"""@SDL OK, but I would hope they'd sign him to ...",0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
4584,"""\n\n Please stop. If you continue to vandaliz...",0.0,0.0,0.0,1.0,0.0
4585,Will you be archiving NWS and SPC maps from to...,0.0,1.0,0.0,0.0,0.0
4586,NPOV tag \nAn anonymous user removed the NPOV ...,0.0,1.0,0.0,0.0,0.0
4587,I have added some introductory material that I...,0.0,1.0,0.0,0.0,0.0


In [ ]:
classes = ['good', 'discussion', 'motivational',	'demotivational',	'irrelavent or abuse']

X = dataset.loc[:, ['Comment']]
y = dataset.loc[:, classes]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1
)

In [ ]:
len(X_train)

3212

### log_reg

In [ ]:
def tokenizer(text):
    return text.split()

In [ ]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)
small_param_grid = [
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [None],
        'vect__tokenizer': [tokenizer, tokenizer_porter],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0]
    },
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer],
        'vect__use_idf':[False],
        'vect__norm':[None],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0]
    },
]
lr_tfidf = Pipeline([
    ('vect', tfidf),
    ('clf', LogisticRegression(solver='liblinear'))
])
gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid,
                            scoring='accuracy', cv=5,
                            verbose=2, n_jobs=1)
gs_lr_tfidf.fit(X_train.Comment, y_train.good)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer_porter 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x7f8a4334b440>,
                                              <function tokenizer_porter at 0x7f8a4334b710>...
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                 

In [ ]:
print(f'Best parameter set : {gs_lr_tfidf.best_params_}')

Best parameter set : {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer_porter at 0x7f8a4334b710>}


In [ ]:
print(f'CV Accuracy: {gs_lr_tfidf.best_score_:.3f}')
clf = gs_lr_tfidf.best_estimator_
print(f'Test Accuracy: {clf.score(X_test.Comment, y_test.good):.3f}')

CV Accuracy: 0.863
Test Accuracy: 0.861


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)
small_param_grid = [
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [None],
        'vect__tokenizer': [tokenizer, tokenizer_porter],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0]
    },
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer],
        'vect__use_idf':[False],
        'vect__norm':[None],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0]
    },
]
lr_tfidf = Pipeline([
    ('vect', tfidf),
    ('clf', LogisticRegression(solver='liblinear'))
])
gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid,
                            scoring='accuracy', cv=5,
                            verbose=2, n_jobs=1)
gs_lr_tfidf.fit(X_train.Comment, y_train.discussion)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.3s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.3s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.3s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.3s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.3s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer_porter 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x7f8a4334b440>,
                                              <function tokenizer_porter at 0x7f8a4334b710>...
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                 

In [ ]:
print(f'Best parameter set : {gs_lr_tfidf.best_params_}')

Best parameter set : {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer_porter at 0x7f8a4334b710>}


In [ ]:
print(f'CV Accuracy: {gs_lr_tfidf.best_score_:.3f}')
clf = gs_lr_tfidf.best_estimator_
print(f'Test Accuracy: {clf.score(X_test.Comment, y_test.discussion):.3f}')

CV Accuracy: 0.752
Test Accuracy: 0.733


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)
small_param_grid = [
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [None],
        'vect__tokenizer': [tokenizer, tokenizer_porter],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0]
    },
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer],
        'vect__use_idf':[False],
        'vect__norm':[None],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0]
    },
]
lr_tfidf = Pipeline([
    ('vect', tfidf),
    ('clf', LogisticRegression(solver='liblinear'))
])
gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid,
                            scoring='accuracy', cv=5,
                            verbose=2, n_jobs=1)
gs_lr_tfidf.fit(X_train.Comment, y_train.motivational)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.1s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer_porter 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x7f8a4334b440>,
                                              <function tokenizer_porter at 0x7f8a4334b710>...
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                 

In [ ]:
print(f'Best parameter set : {gs_lr_tfidf.best_params_}')

Best parameter set : {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x7f8a4334b440>}


In [ ]:
print(f'CV Accuracy: {gs_lr_tfidf.best_score_:.3f}')
clf = gs_lr_tfidf.best_estimator_
print(f'Test Accuracy: {clf.score(X_test.Comment, y_test.motivational):.3f}')

CV Accuracy: 0.942
Test Accuracy: 0.935


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)
small_param_grid = [
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [None],
        'vect__tokenizer': [tokenizer, tokenizer_porter],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0]
    },
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer],
        'vect__use_idf':[False],
        'vect__norm':[None],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0]
    },
]
lr_tfidf = Pipeline([
    ('vect', tfidf),
    ('clf', LogisticRegression(solver='liblinear'))
])
gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid,
                            scoring='accuracy', cv=5,
                            verbose=2, n_jobs=1)
gs_lr_tfidf.fit(X_train.Comment, y_train.demotivational)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.1s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer_porter 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x7f8a4334b440>,
                                              <function tokenizer_porter at 0x7f8a4334b710>...
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                 

In [ ]:
print(f'Best parameter set : {gs_lr_tfidf.best_params_}')

Best parameter set : {'clf__C': 1.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer_porter at 0x7f8a4334b710>}


In [ ]:
print(f'CV Accuracy: {gs_lr_tfidf.best_score_:.3f}')
clf = gs_lr_tfidf.best_estimator_
print(f'Test Accuracy: {clf.score(X_test.Comment, y_test.demotivational):.3f}')

CV Accuracy: 0.783
Test Accuracy: 0.780


In [ ]:
y_train_irrelavent = y_train.iloc[:, -1]
y_test_irrelavente = y_test.iloc[:, -1]

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)
small_param_grid = [
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [None],
        'vect__tokenizer': [tokenizer, tokenizer_porter],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0]
    },
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer],
        'vect__use_idf':[False],
        'vect__norm':[None],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0]
    },
]
lr_tfidf = Pipeline([
    ('vect', tfidf),
    ('clf', LogisticRegression(solver='liblinear'))
])
gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid,
                            scoring='accuracy', cv=5,
                            verbose=2, n_jobs=1)
gs_lr_tfidf.fit(X_train.Comment, y_train_irrelavent)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.1s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f8a4334b440>; total time=   0.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer_porter 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x7f8a4334b440>,
                                              <function tokenizer_porter at 0x7f8a4334b710>...
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                 

In [ ]:
print(f'Best parameter set : {gs_lr_tfidf.best_params_}')

Best parameter set : {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer_porter at 0x7f8a4334b710>}


In [ ]:
print(f'CV Accuracy: {gs_lr_tfidf.best_score_:.3f}')
clf = gs_lr_tfidf.best_estimator_
print(f'Test Accuracy: {clf.score(X_test.Comment, y_test_irrelavente):.3f}')

CV Accuracy: 0.856
Test Accuracy: 0.861


### overall dataset

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
word_vectorizer = TfidfVectorizer(
    strip_accents='unicode',                 
    token_pattern=r'\w{1,}',    
    ngram_range=(1, 3),         
    stop_words = stop,
    sublinear_tf=True)

word_vectorizer.fit(X_train.Comment)

TfidfVectorizer(ngram_range=(1, 3),
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')

In [ ]:
X_train_transformed = word_vectorizer.transform(X_train.Comment)
X_test_transformed = word_vectorizer.transform(X_test.Comment)

In [ ]:
print('X_train_transformed', X_train_transformed.shape)
print('y_train', y_train.shape)
print('X_test_transformed', X_test_transformed.shape)
print('y_test', y_test.shape)

X_train_transformed (3212, 119517)
y_train (3212, 5)
X_test_transformed (1377, 119517)
y_test (1377, 5)


In [ ]:
X_train_transformed

<3212x119517 sparse matrix of type '<class 'numpy.float64'>'
	with 163327 stored elements in Compressed Sparse Row format>

In [ ]:
%%time
pipe_Log_reg =  Pipeline([("Log_reg", LogisticRegression(random_state = 42,max_iter = 10000))])
params = {'Log_reg__penalty': ['l2'],
          'Log_reg__C': [0.5,0.75,1],
          'Log_reg__class_weight': ['balanced',None],
          'Log_reg__solver': ['liblinear']}
Log_reg = GridSearchCV(pipe_Log_reg,params,scoring='roc_auc',return_train_score=True,n_jobs=-1)
Log_reg.fit(X_train_transformed, y_train.good)
print(f'Best_score = {Log_reg.best_score_}')
print(f'Best_params = {Log_reg.best_params_}')
print(f'Roc_auc at test:  = {roc_auc_score(y_test.good, Log_reg.predict(X_test_transformed))}')

Best_score = 0.870319851921249
Best_params = {'Log_reg__C': 0.75, 'Log_reg__class_weight': 'balanced', 'Log_reg__penalty': 'l2', 'Log_reg__solver': 'liblinear'}
Roc_auc at test:  = 0.7557765378427412
CPU times: user 222 ms, sys: 65.4 ms, total: 287 ms
Wall time: 1.99 s


In [ ]:
log_reg = LogisticRegression(C = 1, penalty='l2', solver = 'liblinear',class_weight = 'balanced', random_state=50)

classifier_ovr_log = OneVsRestClassifier(log_reg)
classifier_ovr_log.fit(X_train_transformed, y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1, class_weight='balanced',
                                                 random_state=50,
                                                 solver='liblinear'))

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print("ROC AUC Score Test:", roc_auc_score(y_test, classifier_ovr_log.predict(X_test_transformed)))
print(classification_report(y_test, classifier_ovr_log.predict(X_test_transformed)))

ROC AUC Score Test: 0.6978307349093349
              precision    recall  f1-score   support

           0       0.65      0.60      0.62       262
           1       0.58      0.66      0.62       464
           2       0.33      0.19      0.24        95
           3       0.53      0.47      0.50       320
           4       0.66      0.62      0.64       287

   micro avg       0.59      0.57      0.58      1428
   macro avg       0.55      0.51      0.52      1428
weighted avg       0.58      0.57      0.57      1428
 samples avg       0.52      0.57      0.53      1428



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RF_clf = RandomForestClassifier()
classifier_ovr_log = OneVsRestClassifier(RF_clf)
classifier_ovr_log.fit(X_train_transformed, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier())

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print("ROC AUC Score Test:", roc_auc_score(y_test, classifier_ovr_log.predict(X_test_transformed)))
print(classification_report(y_test, classifier_ovr_log.predict(X_test_transformed)))

ROC AUC Score Test: 0.6375751736931246
              precision    recall  f1-score   support

           0       0.69      0.47      0.56       262
           1       0.64      0.32      0.43       464
           2       1.00      0.09      0.17        95
           3       0.65      0.21      0.32       320
           4       0.91      0.47      0.62       287

   micro avg       0.72      0.34      0.46      1428
   macro avg       0.78      0.31      0.42      1428
weighted avg       0.73      0.34      0.45      1428
 samples avg       0.34      0.34      0.34      1428



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SGDclassifier

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
sgd = SGDClassifier()
classifier_ovr_log = OneVsRestClassifier(RF_clf)
classifier_ovr_log.fit(X_train_transformed, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier())

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print("ROC AUC Score Test:", roc_auc_score(y_test, classifier_ovr_log.predict(X_test_transformed)))
print(classification_report(y_test, classifier_ovr_log.predict(X_test_transformed)))

ROC AUC Score Test: 0.6353399760654319
              precision    recall  f1-score   support

           0       0.68      0.46      0.55       262
           1       0.61      0.30      0.40       464
           2       0.90      0.09      0.17        95
           3       0.63      0.20      0.31       320
           4       0.91      0.49      0.64       287

   micro avg       0.70      0.33      0.45      1428
   macro avg       0.75      0.31      0.41      1428
weighted avg       0.71      0.33      0.44      1428
 samples avg       0.34      0.33      0.33      1428



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
